In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd

sections_7_CS = ['7A','7B','7C','7D','7E']
courses_7_CS = ['FYP-1', 'IS', 'PP', 'ML', 'AI']
combination_7_CS = list(product(sections_7_CS, courses_7_CS))

sections_7_AI = ['7A','7B','7C','7D','7E']
courses_7_AI = ['FYP-1', 'IS', 'PP', 'ML', 'AI']
combination_7_AI = list(product(sections_7_AI, courses_7_AI))

sections_5 = ['5A', '5B', '5C','5D','5E']
courses_5 = ['ALGO', 'GT', 'PDC', 'DB', 'SDA']
combination_5 = list(product(sections_5, courses_5))

sections_3 = ['3A', '3B','3C','3D','3E']
courses_3 = ['LA', 'DS', 'COAL', 'Disc', 'FOM']
combination_3 = list(product(sections_3, courses_3))

sections_1 = ['1A','1B' ,'1C','1D','1E']
courses_1 = ['PF', 'ECC', 'PHY', 'ICT', 'ISL', 'CAL']
combination_1 = list(product(sections_1, courses_1))

# Merged Combinations
combinations_sections_courses = combination_1 + combination_3 + combination_5 + combination_7_CS + combination_7_AI

# Create Bipartite Graph
G = nx.Graph()

# Add nodes and edges to the graph
for section, course in combinations_sections_courses:
    G.add_node(section, bipartite=0)
    G.add_node(course, bipartite=1)
    G.add_edge(section, course)

# Plot the bipartite graph
pos = {node: (0, i) for i, node in enumerate(sections_1 + sections_3 + sections_5 + sections_7_AI + sections_7_CS)}
pos.update({node: (1, i) for i, node in enumerate(courses_1 + courses_3 + courses_5 + courses_7_AI + courses_7_CS)})

nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=800)
plt.title("Bipartite Graph of Sections and Courses")
plt.show()

rooms = ['E1','E2','E3','E4','E5','R109','R12','R11', 'A1', 'A2','A3','A4','A5','A6','A7']
timeslots = ['8-9','9-10','10-11','11-12','12-1','1-2','2-3','3-4']
combination_rooms_slots = list(product(rooms, timeslots))

unique_combinations_set_rooms_slots = set(combination_rooms_slots)
unique_combinations_list_rooms_slots = list(unique_combinations_set_rooms_slots)
G = nx.Graph()
G.add_nodes_from(rooms, bipartite=0)
G.add_nodes_from(timeslots, bipartite=1)
G.add_edges_from(unique_combinations_list_rooms_slots)

all_combinations_by_rooms = [[] for _ in rooms]
for combination in unique_combinations_list_rooms_slots:
    rooms_index = rooms.index(combination[0])
    all_combinations_by_rooms[rooms_index].append(combination)

pos = {node: (0, i) for i, node in enumerate(rooms)}
pos.update({node: (1, i) for i, node in enumerate(timeslots)})

nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=800)
plt.title(f"Bipartite Graph of Rooms and Timeslots")
plt.show()

if len(combinations_sections_courses) >= len(all_combinations_by_rooms):
    length = len(all_combinations_by_rooms)
else:
    length = len(combinations_sections_courses)

# Process each section

week_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

for k in range(5):
    print(f"{week_days[k]}")
    all_matching_set = []
    item = all_combinations_by_rooms.pop(-1)
    all_combinations_by_rooms.insert(0, item)

    for i in range(length):
        timeslots_of_room = all_combinations_by_rooms[i]
        combination_for_bipartite = list(product(timeslots_of_room, combinations_sections_courses))
        unique_combinations_set_for_bipartite = set(combination_for_bipartite)
        unique_combinations_list_for_bipartite = list(unique_combinations_set_for_bipartite)

        G = nx.Graph()
        G.add_nodes_from(timeslots_of_room, bipartite=0)
        G.add_nodes_from(combinations_sections_courses, bipartite=1)
        G.add_edges_from(unique_combinations_list_for_bipartite)
        node_removal_list = []

        for node in G.nodes():
            for dictionary in all_matching_set:
                for key, value in dictionary.items():
                    if node == value:
                        node_removal_list.append(node)

        for node in node_removal_list:
            G.remove_node(node)


        pos = {node: (0, j) for j, node in enumerate(timeslots_of_room)}
        pos.update({node: (1, j) for j, node in enumerate(combinations_sections_courses)})

        if not nx.is_connected(G):
            print("All sections and their courses are matched.")
            break

        max_matching = nx.bipartite.maximum_matching(G)
        midpoint_index = len(max_matching) // 2
        half_max_matching = {key: value for i, (key, value) in enumerate(max_matching.items()) if i < midpoint_index}
        all_matching_set.append(half_max_matching)

        print("Maximum Matching:")
        for rooms_timeslots, course_section in half_max_matching.items():
            print(f"{rooms_timeslots} - {course_section}")
        nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=800)
        nx.draw_networkx_edges(G, pos, edgelist=max_matching.items(), edge_color='red', width=2)
        plt.title("Bipartite Graph with Maximal Matching")
        plt.show()

    print(all_matching_set)
    dataframe = {'Timeslots': timeslots}
    for i in rooms:
        dataframe[i] = 0

    df = pd.DataFrame(dataframe)
    df.set_index('Timeslots', inplace=True)

    for rooms_allotment in all_matching_set:
        for i in rooms_allotment:
            for j in timeslots:
                if j == i[1]:
                    df.loc[i[1], i[0]] = f"{rooms_allotment[i]}"
                    break

    print(df)
    df.to_csv(f"{week_days[k]} Timetable.csv", index=True)

Output hidden; open in https://colab.research.google.com to view.